In [1]:
import json
import pathlib
import re

import pandas as pd

In [2]:
DATA_DIR = pathlib.Path('../data')

# 1. arXiv メタデータの読み込み
[Kaggle の arXiv Dataset](https://www.kaggle.com/datasets/Cornell-University/arxiv/) からダウンロードしたメタデータを読み込みます。  

かなり昔のものかつ広範なカテゴリの論文が含まれているので、ここから絞り込んで使用します。

In [3]:
ARXIV_METADATA_PATH = DATA_DIR / 'arxiv-metadata-oai-snapshot.json'

In [4]:
MONTH_NAME2NUMBER = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}


def read_arxiv_metadata(arxiv_metadata_path: str) -> pd.DataFrame:
    """arxiv-metadata-oai-snapshot.jsonを読み込む"""
    data = []
    with open(arxiv_metadata_path, encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)
            # 最初に提出された日付を使用する
            year, month, day = _extract_date(item['versions'][0]['created'])
            data.append([
                f'{year}-{month}-{day}',
                f'{year}-{month}',
                _preprocess_text(item['title']),
                _preprocess_text(item['abstract']),
                item['categories'].split(' '),
                f'https://arxiv.org/abs/{item["id"]}',
            ])

    cols = ['date', 'year_month', 'title', 'abstract', 'categories', 'url']
    return pd.DataFrame(data, columns=cols)


def _extract_date(text: str) -> tuple[str, str, str]:
    """arXivのメタデータのversionsのcreatedから年月日を取得する"""
    day, month_name, year = text.split(' ')[1:4]
    return year, MONTH_NAME2NUMBER[month_name], f'{day:0>2}'


def _preprocess_text(text: str) -> str:
    """arXivのメタデータのtitleとabstractの空白/改行を削除する"""
    return re.sub(r'\s+', ' ', text.strip())

In [5]:
df_all = read_arxiv_metadata(ARXIV_METADATA_PATH)

print(f'{len(df_all):,}')
df_all.tail(2)

2,396,090


,date,year_month,title,abstract,categories,url
2396088,1996-09-18,1996-09,Density of States and NMR Relaxation Rate in A...,We show that the density of states in an aniso...,"[supr-con, cond-mat.supr-con]",https://arxiv.org/abs/supr-con/9609003
2396089,1996-09-25,1996-09,Ginzburg Landau theory for d-wave pairing and ...,The Ginzburg Landau theory for d_{x^2-y^2}-wav...,"[supr-con, cond-mat.supr-con]",https://arxiv.org/abs/supr-con/9609004


# 2. 期間とカテゴリの絞り込み
日付が2022年から2023年の間、カテゴリが`cs`もしくは`stat`のもののみを残します。  
（2021年以前には LLM 関連の論文がほぼなかったため除外しています。）  

このデータを分析元の arXiv データとして扱います。   

In [6]:
SAVE_PATH = DATA_DIR / 'papers.csv'

In [7]:
YEAR_MONTH_COL = 'year_month'
CATEGORIES_COL = 'categories'
YEAR_MONTH_RANGE = ('2022-01', '2023-12')
TARGET_CATEGORIES = ['cs', 'stat']


def squeeze_by_year_month_and_categories(df: pd.DataFrame) -> pd.DataFrame:
    """読み込んだメタデータからデータを絞り込む"""
    cond_year_month = (df[YEAR_MONTH_COL] >= min(YEAR_MONTH_RANGE)) & (df[YEAR_MONTH_COL] <= max(YEAR_MONTH_RANGE))
    regex = re.compile(rf'^({"|".join(TARGET_CATEGORIES)})')
    cond_categories = df[CATEGORIES_COL].apply(lambda cats: any(regex.search(cat) for cat in cats))

    return df[cond_year_month & cond_categories]

In [8]:
df = squeeze_by_year_month_and_categories(df_all)

print(f'{len(df):,}')
print(f'{df.date.min()} ~ {df.date.max()}')

191,185
2022-01-01 ~ 2023-12-31


In [9]:
df.to_csv(SAVE_PATH, header=True, index=False)